In [33]:
# https://www.dropbox.com/developers/apps/info/pu644k33a199294#settings

import pandas as pd
import numpy as np
import dropbox
import camelot
import tempfile
import fitz
import os
import re

In [34]:
def contains(string:str, contains:list) -> bool:
    for cont in contains:
        if cont in string:
            return True
    
    return False

## Classify

In [35]:
def get_content(extension, file_obj):
    if extension == ".pdf":
        reader = fitz.open(stream=file_obj)
        return reader.load_page(0).get_text()
    elif extension == ".xlsx":
        return pd.read_excel(file_obj).to_string()
    elif extension == ".xlsb":
        return pd.read_excel(file_obj, engine='pyxlsb').to_string()
    else:
        return None


def classify_file(path, file_obj, verbose=False):
    try:
        extension = os.path.splitext(path)[1]
        content = get_content(extension, file_obj)

        content = content.lower()
        if "wrapbook" in content:
            return "OTHER"
        elif contains(content, ["cost summary", "hot budget", "film production cost summary"]):
            return "CS"
        elif "purchase order" in content:
            return "PO"
        elif "payroll" in content:
            return "PR"
        else:
            return "OTHER"
    except Exception as e:
        print("classification error %s at: " % e, path) if verbose else None
        return "OTHER"

## Department Getter

In [36]:
def get_dept_from_line(ln:int) -> str:
    try:
        ln = int(ln)
    except ValueError:
        return ln

    if ln in range(51):
        return "PRE-PRODUCTION | WRAP LABOR"
    elif ln in range(51, 101):
        return "SHOOTING LABOR"
    elif ln in range(101, 114):
        return "PRE-PRODUCTION | WRAP EXPENSES"
    elif ln in range(114, 140):
        return "LOCATION AND TRAVEL"
    elif ln in range(140,151):
        return "MAKEUP, WARDROBE, AND ANIMALS"
    elif ln in range(151, 168):
        return "STUDIO | STAGE RENTAL / EXPENSES"
    elif ln in range(168,181):
        return "ART DEPARTMENT LABOR"
    elif ln in range(181, 193):
        return "ART DEPARTMENT EXPENSES"
    elif ln in range(193, 211):
        return "EQUIPMENT COSTS"
    elif ln in range(211, 217):
        return "FILMSTOCK, DEVELOP AND PRINT"
    elif ln in range(217,227):
        return "MISCELLANEOUS"
    elif ln in range(227, 234):
        return "DIRECTOR | CREATIVE FEES"
    elif ln in range(234, 271):
        return "TALENT LABOR"
    elif ln in range(271, 277):
        return "TALENT EXPENSES"
    elif ln in range(277, 282):
        return "POST PRODUCTION LABOR"
    elif ln in range(282, 330):
        return "EDITORIAL | FINISHING | POST PRODUCTION"
    else:
        return "OTHER"

In [37]:
def camelot_read_pdf_bytes(file_obj, table_num=0) -> pd.DataFrame:
    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
        temp_pdf.write(file_obj)
        return camelot.read_pdf(temp_pdf.name)._tables[table_num].df.copy()

## Cost Summary Reader

In [38]:
HB_CS_COLS = ["SECTION", "drop", "BID TOTALS", "ACTUAL", "VARIANCE"]


def read_hot_budget_cs(file_obj, extension) -> pd.DataFrame:
    if extension == ".pdf":
        _df = camelot_read_pdf_bytes(file_obj, 1)
        
        _df.drop(12, inplace=True)

        _df.columns = HB_CS_COLS
        _df.drop(columns=["drop"], inplace=True)
        _df = _df.loc[1:]

        _df = _df.replace([r"CS\d+\b ", r".*\n", "\)", ","], "", regex=True).replace("\(", "-", regex=True)

        _df[_df.columns[1:]] = _df.iloc[:, 1:].replace("", np.nan).astype(float)

        _df = _df.dropna(thresh=2)

        return _df.reset_index(drop=True)
    elif extension == ".xlsx":
        _df = pd.read_excel(file_obj, skiprows=10, nrows=23).dropna(how="all", axis=1).drop(11).dropna(thresh=3).rename(columns={"ESTIMATED COST SUMMARY":"SECTION"})
        _df.drop(_df.columns[1], axis=1, inplace=True)

        sep_nums = lambda x: x[re.search(r"\d ", x).end():]
        _df.SECTION = _df.SECTION.apply(sep_nums)

        return _df.reset_index(drop=True)
    else:
        return pd.DataFrame()


def read_GetActual_cs(file_obj) -> pd.DataFrame:
    reader = fitz.open(stream=file_obj)
    content = reader.load_page(0).get_text()

    start = re.search(r"\b[A-Z]\s", content[2:]).start()
    content = re.sub(r"\b[A-Z]\s|Bid Actual|\,|\)", "", content.replace("(", "-"))
    content = content[start:content.find("\nGRAND TOTAL")].split("\n")
    _df = pd.DataFrame(columns=["SECTION", "BID TOTALS", "ACTUAL"])
    
    for line in content:
        vals = line.split("$")
        if len(vals) > 1:
            _df.loc[len(_df)] = vals[:3]

    _df[["BID TOTALS", "ACTUAL"]] = _df[["BID TOTALS", "ACTUAL"]].astype(float)
    _df = _df.drop(_df[_df.SECTION.str.contains("SUB TOTAL")].index)

    _df["VARIANCE"] = _df["ACTUAL"] - _df["BID TOTALS"]
    _df.SECTION = _df.SECTION.apply(str.strip)

    return _df


def read_cost_summary(file_obj, extension) -> pd.DataFrame:
    content = get_content(extension, file_obj)
    
    if "ESTIMATED COST SUMMARY" in content:
        return read_hot_budget_cs(file_obj, extension)
    elif "Film Production Cost Summary" in content:
        return read_GetActual_cs(file_obj)

## Payroll Reader

In [39]:
def read_sheet(file_obj, extension) -> pd.DataFrame:
    if extension == ".xlsx":
        _df = pd.read_excel(file_obj, header=4)
    if extension == ".xlsb":
        _df = pd.read_excel(file_obj, engine='pyxlsb', header=4)
    
    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)
    if "RATE" in _df.columns:
        _df.RATE = _df.RATE.astype(float)
    
    return _df

In [40]:
PR_COLS = ['LINE', 'PAYEE', 'PO', 'F1', 'F2', 'DAYS', 'RATE', 'BASE', '1.5', '2', '3', 'TAXABLE', 'NON-TAX', 'TOTAL ST', 'TOTAL OT', 'ACTUAL', 'FRINGE 1', 'FRINGE 2', 'LINE DESCRIPTION']

def read_pdf_payroll(file_obj) -> pd.DataFrame:
    _df = camelot_read_pdf_bytes(file_obj, 0)
    
    _df.columns = PR_COLS
    _df = _df.iloc[1:].reset_index(drop=True).replace("", np.nan).dropna(how="all")

    _df.LINE.fillna(_df.PAYEE, inplace=True)
    _df[['LINE', 'PAYEE']] = _df.LINE.str.split(" ", n=1, expand=True)

    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)

    return _df

In [41]:
def read_payroll(file_obj, extension) -> pd.DataFrame:
    if extension == ".pdf":
        _df = read_pdf_payroll(file_obj)
    elif extension in [".xlsx", "xlsb"]:
        _df = read_sheet(file_obj, extension)
    else:
        return pd.DataFrame()
    
    _df = _df.dropna(how="all")
    _df.RATE = _df.RATE.astype(float)
    _df.DAYS = _df.DAYS.astype(float)

    _df["EST"] = _df.RATE * _df.DAYS
    _df["VARIANCE"] = _df.ACTUAL - _df.EST
    _df["VAR_PCT"] = _df.VARIANCE / _df.EST * 100
    _df["SECTION"] = _df.LINE.apply(get_dept_from_line)

    return _df

## Purchase Order Log Reader

In [42]:
PO_COLS = ["LINE", "PAYEE", "PO", "DATE", "PAYID", "ACTUAL", "LINE DESCRIPTION"]


def read_pdf_purchase_order(file_obj) -> pd.DataFrame:
    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
        temp_pdf.write(file_obj)
        _df = camelot.read_pdf(temp_pdf.name)._tables[0].df.copy()
    
    _df.columns = PO_COLS
    _df = _df.iloc[1:].reset_index(drop=True).replace("", np.nan).dropna(how="all")

    _df.LINE.fillna(_df.PAYEE, inplace=True)
    _df[['LINE', 'PAYEE']] = _df.LINE.str.split(" ", n=1, expand=True)

    _df.ACTUAL.fillna(_df["LINE DESCRIPTION"], inplace=True)
    _df[['ACTUAL', 'LINE DESCRIPTION']] = _df.ACTUAL.str.split(" ", n=1, expand=True)

    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)

    return _df

In [43]:
def read_sheet(file_obj, extension) -> pd.DataFrame:
    if extension == ".xlsx":
        _df = pd.read_excel(file_obj, header=4)
    if extension == ".xlsb":
        _df = pd.read_excel(file_obj, engine='pyxlsb', header=4)
    
    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)
    if "RATE" in _df.columns:
        _df.RATE = _df.RATE.astype(float)
    
    return _df

In [44]:
def read_purchase_order(file_obj, extension) -> pd.DataFrame:
    if extension == ".pdf":
        _df = read_pdf_purchase_order(file_obj)
    elif extension in [".xlsx", ".xlsb"]:
        _df = read_sheet(file_obj, extension)
    else:
        return pd.DataFrame()
    
    _df = _df.dropna(how="all")
    

    return _df

## Dropbox

In [45]:
# Expires frequently
DB_TOKEN = "sl.Bgtwyk3MBTDG9uqeJppkB8YV_LxH8_XDlGQEfte9uZ7G5tPqwuv9WQYtMJvX0hOkswB0WCdC4ENTP1-2ansFMsUWNXyQmgwp9Z4O5irbxJFtXY75cZXbHdUhDupGRi6AeaspRtk"

dbx = dropbox.Dropbox(DB_TOKEN)

In [46]:
from io import BytesIO

class DbxDataRetriever:
    datasets = {
        "CS" : [],
        "PR" : [],
        "PO" : []
    }

    def __init__(self, link, dbx) -> None:
        self.path = self.path_from_link(link)
        self.dbx = dbx

    def path_from_link(self, path):
        start_key = "sh/"

        if start_key in path:
            end = path.find("?")
        else:
            start_key = "home/"
            end = len(path)
        
        start = path.find(start_key) + len(start_key) - 1
        return path[start : end]
    
    def get_content_and_type(self, dbx_path):
        _meta, res = dbx.files_download(dbx_path)
        file_obj = res.content
        _type = classify_file(dbx_path, file_obj, verbose=True)

        return _type, file_obj

    def file_to_df(self, _type:str, path:str, file_obj:bytes) -> pd.DataFrame:
        extension = os.path.splitext(path)[1]

        if _type == "CS":
            return read_cost_summary(file_obj, extension)
        elif _type == "PR":
            return read_payroll(file_obj, extension)
        elif _type == "PO":
            return read_purchase_order(file_obj, extension)
        else:
            return pd.DataFrame()

    def gen_data(self):
        res = dbx.files_list_folder(self.path)

        for entry in res.entries:
            current_path = entry.path_display

            if isinstance(entry, dropbox.files.FileMetadata):
                project_name = current_path.split("/")[-3]
                _type, file_obj = self.get_content_and_type(current_path)
                # self.datasets[_type].append(file_obj)
                print(current_path)
                df = self.file_to_df(_type, current_path, file_obj)
                print(df)
                # if not df.empty:
                #     df["PROJECT NAME"] = project_name
                #     print(df)
                
            elif isinstance(entry, dropbox.files.FileMetadata):
                pass




In [47]:
link = "https://www.dropbox.com/home/_JOB_ACTUALS/22003_EA/Purchase Order Log"

dbx = dropbox.Dropbox(DB_TOKEN)


retr = DbxDataRetriever(link, dbx)

retr.gen_data()

/_JOB_ACTUALS/22003_EA/Purchase Order Log/22003_EAPOLog.xlsx
   LINE                              PAYEE        PO       DATE  PAYID  \
0   280                   Assistant Editor  22003-22        NaT    TBP   
1   181             Austin Barbera - Venmo  22003-16 2022-02-08   4000   
2   181                        Baja Fresh   22003-23 2022-02-05   7951   
3   181                        Car Rental   22003-21 2022-02-05   7951   
4    77  Chadrick Fellers - Video Engineer  22003-07 2022-02-11    TBP   
..  ...                                ...       ...        ...    ...   
56  NaN                                NaN       NaN        NaT    NaN   
57  NaN                                NaN       NaN        NaT    NaN   
58  NaN                                NaN       NaN        NaT    NaN   
59  NaN                                NaN       NaN        NaT    NaN   
60  NaN                                NaN       NaN        NaT  TOTAL   

      ACTUAL  LINE DESCRIPTION  Unnamed: 7  
0    

In [48]:
# payroll_dfs = []
# cs_dfs = []

# dbx = dropbox.Dropbox(DB_TOKEN)


# def download_folder_contents(directory_path):
#     result = dbx.files_list_folder(directory_path, recursive=True)
#     dir_name = os.path.basename(directory_path)
#     dir_files = {}
    
#     for entry in result.entries:
#         current_path = entry.path_display

#         if isinstance(entry, dropbox.files.FileMetadata):
#             # Download the file
#             file_name = os.path.basename(current_path)
#             meta, res = dbx.files_download(current_path)
#             content = res.content
#         elif isinstance(entry, dropbox.files.FolderMetadata):
#             # Recursively download contents of subfolder
#             download_folder_contents(current_path)



# def download(dbx, folder, subfolder, name):
#     """Download a file.
#     Return the bytes of the file, or None if it doesn't exist.
#     """
#     path = '/%s/%s/%s' % (folder, subfolder.replace(os.path.sep, '/'), name)
#     while '//' in path:
#         path = path.replace('//', '/')

#     try:
#         md, res = dbx.files_download(path)
#     except dropbox.exceptions.HttpError as err:
#         print('*** HTTP error', err)
#         return None
    
#     data = res.content
#     print(len(data), 'bytes; md:', md)
#     return data

In [49]:
# start_dir = "data/JOB_ACTUALS"

# payroll_dfs = []
# cs_dfs = []


# for directory in os.listdir(start_dir):
#     dir_path = os.path.join(start_dir, directory)
#     good = True
#     dir_files = {}

#     if os.path.isdir(dir_path):
#         for file in os.listdir(dir_path):
#             file_path = os.path.join(dir_path, file)
#             _type = classify_file(file_path)
#             if not _type in dir_files:
#                 dir_files[_type] = []
            
#             dir_files[_type].append(file_path)
    
#     for _type in ["PO", "PR", "CS"]:
#         if not _type in dir_files:
#             good = False
    
#     if dir_files.get("PR"):
#         for pr_path in dir_files.get("PR"):
#             df = read_payroll(pr_path)
#             df["PROJECT_NAME"] = directory
#             payroll_dfs.append(df)
    
    # if dir_files.get("CS"):
    #     for path in dir_files.get("CS"):
    #         df = read_cost_summary(path)
    #         df["PROJECT_NAME"] = directory
    #         cs_dfs.append(df)

#     if good:
#         print(directory)
        
    


In [50]:
# _df = pd.concat(payroll_dfs)
# _df.RATE = _df.RATE.astype(float)
# _df.DAYS = _df.DAYS.astype(float)

# _df["EST"] = _df.RATE * _df.DAYS
# _df["VARIANCE"] = _df.ACTUAL - _df.EST
# _df["VAR_PCT"] = _df.VARIANCE / _df.EST * 100
# _df["SECTION"] = _df.LINE.apply(get_dept_from_line)


In [51]:
# _df.PROJECT_NAME.unique()

In [52]:
# # fig, ax = plt.subplots(figsize=(10, 10))

# BY = "SECTION"
# FOR = "VARIANCE"

# pr_grouped = _df.groupby(BY).mean(numeric_only=True).sort_values(FOR, ascending=True).query("%s > 0" % FOR)[FOR]

# # pr_grouped.plot(kind="barh", ax=ax)

# # ax.set_title("Payee's Going Over Budget")
# # ax.set_ylabel("Payee")
# # ax.set_xlabel("Amount Over Budget ($)")
# pr_grouped

In [53]:
# cs_combined = pd.concat(cs_dfs)

# cs_grouped = cs_combined.groupby("SECTION").mean(numeric_only=True).sort_values("VARIANCE", ascending=False).round(2)
# cs_grouped["VAR_PCT"] = (cs_grouped.VARIANCE / (cs_grouped["BID TOTALS"] + 1e-3)).round(2) * 100

# cs_grouped